In [6]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import pandas as pd

In [20]:
otu_size = 717
latent_size = 10
hidden_layers = [128, 64]
encoder = keras.Sequential(name = 'encoder')
encoder.add(layers.Input(otu_size, name = 'otu_input'))
for size in hidden_layers:
    encoder.add(layers.Dense(size, activation=layers.ReLU(), name = 'encoder_{}'.format(size)))
encoder.add(layers.Dense(latent_size, name = 'latent'))
encoder.build()
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_128 (Dense)         (None, 128)               91904     
                                                                 
 encoder_64 (Dense)          (None, 64)                8256      
                                                                 
 latent (Dense)              (None, 10)                650       
                                                                 
Total params: 100,810
Trainable params: 100,810
Non-trainable params: 0
_________________________________________________________________


In [21]:
decoder = keras.Sequential(name = 'decoder')
decoder.add(layers.Input(latent_size, name = 'decoder_input'))
for size in reversed(hidden_layers):
    decoder.add(layers.Dense(size, activation = layers.ReLU(), name = 'decoder_{}'.format(size)))
decoder.add(layers.Dense(otu_size, name = 'decoder_output'))
decoder.build()
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_64 (Dense)          (None, 64)                704       
                                                                 
 decoder_128 (Dense)         (None, 128)               8320      
                                                                 
 decoder_output (Dense)      (None, 717)               92493     
                                                                 
Total params: 101,517
Trainable params: 101,517
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = keras.Model(inputs=layers.Input(otu_size, name = 'otu_input'),
                            outputs=layers.Dense(otu_size, name = 'decoder_output'),
                            name='model')